In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import classifier as cf
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
traindata = pd.read_csv("data/processed_training_data.csv", nrows=10000)

In [3]:
def give_score(row):
    if row["booking_bool"] > 0:
        return int(5)
    if row["click_bool"] > 0:
        return int(1)
    return int(0)

In [4]:
traindata["score"] = traindata.apply(give_score, axis=1)
print(len(traindata))

10000


In [5]:
testdata = pd.read_csv("data/processed_testing_data.csv", nrows=5000)
print(len(testdata))

5000


In [6]:
print(traindata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool', 'score'],
      dtype='object')


In [7]:
print(testdata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'random_bool', 'score'],
      dtype='object')


In [8]:
attrs = ["prop_starrating","prop_review_score" ,"prop_location_score1", 
         "prop_log_historical_price", "price_usd", "srch_length_of_stay"]
flag = "True"
y = traindata["score"]
X = traindata[attrs]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5, random_state=101)

In [9]:
gnb = cf.gaussian_naive_bayes(Xtrain, Xtest, ytrain, ytest, flag)

Gaussian Naive Bayes: cross vali score 0.9394070717424012 , accuracy score 0.9392


In [10]:
dt = cf.decision_tree(Xtrain, Xtest, ytrain, ytest, flag)

Decision Tree: cross vali score 0.9055990121858386 , accuracy score 0.8986


In [11]:
knn = cf.k_nearest_neighbours(Xtrain, Xtest, ytrain, ytest, flag)

K Nearest Neighbours: cross vali score 0.9546007084929241 , accuracy score 0.9538


In [12]:
sgd = cf.stochastic_gradient_descent(Xtrain, Xtest, ytrain, ytest, flag)

Stochastic Gradient Descent: cross vali score 0.9532011052969137 , accuracy score 0.9538


In [13]:
lr = cf.logistic_regression(Xtrain, Xtest, ytrain, ytest, flag)

C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Logistic Regression: cross vali score 0.9548003092913273 , accuracy score 0.9538


C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
def calculate_dcg(scores):
    return np.sum(scores / np.log2(np.arange(len(scores)) + 2))

def calculate_ndcg(score):
    ordered_score = sorted(score, reverse=True)
    dcg = calculate_dcg(score)
    idcg = calculate_dcg(ordered_score)
    return dcg/idcg

In [84]:
"""Metrics to compute the model performance."""

import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer


def dcg_score(y_true, y_score, k=5):
    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):

    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

In [93]:
def output_prediction(testdata, ypred):
    testdata["score"] = ypred.tolist()
    
    df = pd.DataFrame({"srch_id": testdata["srch_id"].values,
                  "prop_id": testdata["prop_id"]})
    a = testdata["srch_id"]
    b = testdata["prop_id"]
    print(ndcg_score(a, b))
    return df

In [94]:
gnbpred = output_prediction(testdata, gnb)
gnbpred

0.0


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [95]:
dtpred = output_prediction(testdata, dt)
dtpred

0.0


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [96]:
knnpred = output_prediction(testdata, knn)
knnpred

0.0


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [97]:
sgdpred = output_prediction(testdata, sgd)
sgdpred

0.0


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [32]:
lrpred = output_prediction(testdata, lr)
lrpred

nan


C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611


In [33]:
xgb = cf.xg_boost(Xtrain, Xtest, ytrain, ytest, flag)
xgb

C:\Users\Helia\anaconda3\lib\site-packages\cupy\_environment.py:211: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
C:\Users\Helia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:20] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Helia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:21] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Helia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\Helia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:28] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Helia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:28] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost: cross vali score 0.9528013042983101 , accuracy score 0.9512


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [34]:
xgbpred = output_prediction(testdata, xgb)
xgbpred

0.18661115943390075


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
4995,321,75719
4996,321,77067
4997,321,81826
4998,321,98611
